In [2]:
using Plots
using Statistics
using LinearAlgebra
using JSON

include("readclassjson.jl");

## 4.600 Sensor integrity monitor. 
A matrix $B \in \mathbb{R}^{k \times m}$ is called an integrity monitor if the following holds:
    
* $B y=0$ for any $y$ which is consistent.
* $B y \neq 0$ for any $y$ which is inconsistent.

If we find such a matrix $B,$ we can quickly check whether $y$ is consistent; we can send an alarm if $B y \neq 0 .$ Note that the first requirement says that every consistent $y$ does not trip the alarm; the second requirement states that every inconsistent $y$ does trip the alarm. Finally, the problem. Find an integrity monitor $B$ for the matrix $A=\left[\begin{array}{ccc}1 & 2 & 1 \\ 1 & -1 & -2 \\ -2 & 1 & 3 \\ 1 & -1 & -2 \\ 1 & 1 & 0\end{array}\right]$ Your $B$ should have the smallest $k$ ( $i . e .,$ number of rows ) as possible. As usual, you have to explain what you're doing, as well as giving us your explicit matrix $B$. You must also verify that the matrix you choose satisfies the requirements.

In [3]:
A = [1 2 1; 1 -1 -2 ; -2 1 3; 1 -1 -2; 1 1 0]

5×3 Array{Int64,2}:
  1   2   1
  1  -1  -2
 -2   1   3
  1  -1  -2
  1   1   0

* When we perform a full QR decomposition of the matrix A, we are left with $A=\left[\begin{array}{ll} Q_{1} & Q_{2} \end{array}\right]\left[\begin{array}{c} R_{1} \\ 0 \end{array}\right]$. The matrix product of $Q_1$ and $R_1$ fully reconstructs A while $Q_2$ is an orthogonal complement to $Q_1$ such that $R(Q_1) \perp R(Q_2)$. 
* Therefore any output generated via the matrix $Q_1$ will lie in the range of $Q_1$ and will also be orthogonal to any vector in the range of $Q_2$. From these facts, it seems clear what needs to be done. Since $Ax = Q_1R_1x$ then $R(A) = R(Q_1R_1) \subset R(Q_1)$. Any y in $R(Ax)$ is also in $R(Q_1)$ and is orthogonal to $R(Q_2)$. 
* If we write this as $y \perp Q_2x \implies Q_2^Ty \perp Q_2^TQ_2x \implies Q_2^Ty \perp x$. 
* Therefore $Q_2^T$ is a suitable B. Note B is has the minimum number of rows needed to produce 0 for any $y \in R(Q_1)$ since $R(Q_1) \perp R(Q_2)$ and $R(Q_1) + R(Q_2)$ span $R^m = R^5$. This allows us to capture every y either consistent with $Ax$ (in $R(Q_1)$) or inconsistent (in $R(Q_2)$) in all of $R^5$.

In [24]:
F = qr(A);

Extract pieces of QR

In [53]:
R1 = F.R
Q1 = F.Q[:,1:3]
Q2 = F.Q[:,4:5];

Create example y

In [73]:
x = 1:3
y = A * x
rank(hcat(A, y)) == rank(A)

true

In [61]:
B = transpose(Q2)

2×5 Transpose{Float64,Array{Float64,2}}:
  0.0894036  -0.1348    0.459174    0.868262   0.0954814
 -0.536989   -0.112889  0.0502894  -0.0801713  0.830628

For a consistent $y = Ax$, $By$ outputs zero.

In [74]:
abs(sum(B * y)) < 1e-15

true

For an inconsistent y, B does not output 0

In [75]:
y[3] = 1
rank(hcat(A, y)) == rank(A)

false

In [76]:
B * y

2-element Array{Float64,1}:
 -3.673389302509566
 -0.4023148392043048